In [3]:
import mysql.connector
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker for generating random data
fake = Faker()

# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "Sama@1234",  # Ensure credentials are correct
    "database": "crop_management"
}

try:
    # Connect to MySQL
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Create crops table if it does not exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS crops (
        id INT AUTO_INCREMENT PRIMARY KEY,
        crop_name VARCHAR(255) NOT NULL,
        planting_date DATE NOT NULL,
        harvest_date DATE NOT NULL,
        growth_stage VARCHAR(50) NOT NULL,
        pest_control_measures TEXT NOT NULL,
        yield_prediction INT NOT NULL
    )
    """)
    conn.commit()

    # List of sample crop names
    crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

    # List of possible growth stages
    growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

    # List of sample pest control measures
    pest_control_measures_list = [
        "Use of organic pesticides",
        "Crop rotation",
        "Neem oil application",
        "Biological pest control",
        "Chemical pesticides",
        "Regular field monitoring",
    ]

    # Function to generate random data for crops
    def generate_data():
        crop_name = random.choice(crop_names)
        planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
        harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
        growth_stage = random.choice(growth_stages)
        pest_control = random.choice(pest_control_measures_list)
        yield_prediction = random.randint(500, 5000)  # Yield in kg

        return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

    # Batch insert records in chunks
    batch_size = 10000  # Insert 10,000 at a time for efficiency
    total_records = 20000000  # Insert 20 million records

    for i in range(0, total_records, batch_size):
        data_batch = [generate_data() for _ in range(batch_size)]
        
        try:
            # Execute batch insert
            cursor.executemany("""
                INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, data_batch)
            conn.commit()
            print(f"{i + batch_size} records inserted...")

        except mysql.connector.Error as e:
            print(f"❌ Error inserting batch at {i}: {e}")
            break  # Stop execution if an error occurs

    print("✅ Data insertion completed successfully!")

except mysql.connector.Error as e:
    print(f"❌ Database connection error: {e}")

finally:
    # Close the database connection safely
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


KeyboardInterrupt: 